In [ ]:
import pandas as pd
from gurobipy import *
import openpyxl
from pprint import pprint

In [ ]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Excel_to_Gurobi_JUP_Network_Model.xlsx")
main = wb['Sample-4']

In [ ]:
# read objective
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')

In [ ]:
# new
# get variable name

column_name = []
for c in range(2,10000):
    v = main.cell(column=c, row=5).value
    if v is None or v == 'Supply':
        break
    elif type(v) != str:
        column_name.append(str(v))            
    else:
        column_name.append(v)
        
n_column = len(column_name)

In [ ]:
# get variable name

column_name = []
for c in range(2,10000):
    v = main.cell(column=c, row=5).value
    if v is None or v == 'Supply':
        break
    elif type(v) != str:
        column_name.append(str(v))            
    else:
        column_name.append(v)
        
n_column = len(column_name)


row_name = []
for c in range(6,10000):
    v = main.cell(column=1, row=c).value
    if v is None or v == 'Demand':
        break
    elif type(v) != str:
        row_name.append(str(v))            
    else:
        row_name.append(v)
        
n_row = len(row_name)


n_variables = n_column * n_row


variable_name = []
for i in range(0,n_row):
    for j in range(0,n_column):
        v_name = row_name[i] + '_TO_' + column_name[j]
        variable_name.append(v_name)

In [ ]:
# get obj coefficient        

obj_coeff=[]

for i in range(0,n_row):
    for j in range(0,n_column):
        v = main.cell(column=j+2, row=i+6).value
        if v is None:
            if obj == -1:
                v = -99999999
            else:
                v = 99999999
        obj_coeff.append(v)

In [ ]:
#read variable properties     
        
variable_type = []
v = main.cell(column=2, row=2).value

for c in range(n_variables):
    if not v:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)


In [ ]:
# get constraints

# get constraints of supply

column_supply = 0

for c in range(2,10000):
    v = main.cell(column=c, row=5).value
    if v == 'Supply':
        column_supply = c
        break

if column_supply == 0:
    print('ERROR! Please insert a column named [Supply] which indicate the constraints of supply')

    
v_supply = []
for i in range(n_row):
    v = main.cell(column=column_supply, row=i+6).value
    v_supply.append(v)
    

values = []
for i in range(n_row):
    r_values = [0] * (n_variables + 2)
    for j in range(n_column):
        r_values[i*n_column + j] = 1
    r_values[n_variables] = '<='
    r_values[n_variables + 1] = v_supply[i]
    if v_supply[i] is not None:
        values.append(r_values)

constr_supply = values

In [ ]:
# get constraints of demand

row_demand = 0

for i in range(n_row+6, 9999):
    v = main.cell(column=1, row=i).value
    if v == 'Demand':
        row_demand = i
        break

if row_demand == 0:
    print('ERROR! Please insert a row named [demand] which indicate the constraints of demand')
    
v_demand = []
for i in range(n_column):
    v = main.cell(column=i+2, row=row_demand).value
    v_demand.append(v)

values = []

for i in range(n_column):
    r_values = [0] * (n_variables + 2)
    for j in range(n_row):
        r_values[i+ j*n_column] = 1
    r_values[n_variables] = '>='
    r_values[n_variables + 1] = v_demand[i]
    if v_demand[i] is not None:
        values.append(r_values)

constr_demand = values

In [ ]:
# get constraints of non-neg

values = []

v = main.cell(column=2, row=3).value

if v == 'Y':
    values = []
    for i in range(n_variables):
        r_values = [0] * (n_variables + 2)
        r_values[i] = 1
        r_values[n_variables] = '>='
        r_values[n_variables + 1] = 0
        values.append(r_values)

constr_non_neg = values

In [ ]:
# get constraints of in_n_out

# get the column name of transshipment node

v_transfer_node_column = []

for i in range(len(v_demand)):
    if v_demand[i] is None:
        v = main.cell(column=i+2, row=5).value
        temp = [v,i]
        v_transfer_node_column.append(temp)

n_transfer_node_column = len(v_transfer_node_column)


# get the row name of transshipment node

v_transfer_node_row = []

for i in range(len(v_supply)):
    if v_supply[i] is None:
        v = main.cell(column=1, row=i+6).value
        temp = [v,i]
        v_transfer_node_row.append(temp)

n_transfer_node_row = len(v_transfer_node_row)

if n_transfer_node_column != n_transfer_node_row:
    print('ERROR! Please check the column name and row name. Some items are missing or redundance.')


# get the coordinate of transshipment node

v_transfer_node = []

for i in range(len(v_transfer_node_column)):
    a = len(v_transfer_node)
    for j in range(len(v_transfer_node_row)):
        if v_transfer_node_row[j][0] == v_transfer_node_column[i][0]:
            temp = [v_transfer_node_column[i][0],[v_transfer_node_column[i][1],v_transfer_node_row[j][1]]]
            v_transfer_node.append(temp)
    b = len(v_transfer_node)
    if a == b:
        print('ERROR! Please check the column name and row name. Some items are missing or redundance.')


# constraints of in_n_out

values = []

n_v_transfer_node = len(v_transfer_node)

for i in range(n_v_transfer_node):
    r_values = [0] * (n_variables + 2)
    transfer_node_column = v_transfer_node[i][1][0]
    transfer_node_row = v_transfer_node[i][1][1]
    
    for j in range(n_row):
        r_values[j*n_column + transfer_node_column] = 1
        r_values[transfer_node_row*n_column + j] = -1
    r_values[n_variables] = '='
    r_values[n_variables + 1] = 0
    values.append(r_values)
    
constr_in_n_out = values

In [ ]:
## read constraints as dataframe

cons = constr_in_n_out + constr_supply  + constr_demand

v = main.cell(column=2, row=3).value

if v == 'Y':
    cons+= constr_non_neg
    
cons_df = pd.DataFrame()

cons_df = pd.DataFrame(cons)

cons_df.rename(columns={n_variables:'constraint type', 
                        n_variables+1:'RHS values'}, inplace=True)

cons_df

In [ ]:
# setup model
m = Model()

In [ ]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
#    x[i].setAttr('lb', variable_lb[i])
#    x[i].setAttr('ub', variable_ub[i])

In [ ]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [ ]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['constraint type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['constraint type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['constraint type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i])

In [ ]:
## run the model
m.optimize()

In [ ]:
# print optimal objective value
m.objVal

In [ ]:
# print optimal decisions (if not listed, that means their optimal value = 0)
m.printAttr('X')